# 🎯 互动实验：Agent记忆优化

## 🧪 实验目标

在这个Notebook中，你将深入实验Agent的记忆管理，包括：
- 测试不同记忆策略的效果
- 优化记忆窗口大小
- 实现关键信息提取
- 对比不同策略的Token消耗

## 🚀 开始实验

首先，让我们导入必要的库并设置环境。

In [1]:
# 导入基础库
import os
import sys
import dotenv
from openai import OpenAI
import json
import re
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd

# 添加项目根目录到 Python 路径，以便导入模块
# 获取 stage1-basic 目录的绝对路径
# 在 Jupyter Notebook 中，工作目录通常是项目根目录或 notebook 目录
# 我们需要找到 stage1-basic 目录
import pathlib

# 方法1：尝试从当前工作目录向上查找
cwd = pathlib.Path.cwd()
stage1_dir = None

# 如果当前在 notebook 目录，向上找到 stage1-basic
if cwd.name == 'notebook':
    stage1_dir = cwd.parent
elif 'stage1-basic' in str(cwd):
    # 如果已经在 stage1-basic 目录
    stage1_dir = cwd if cwd.name == 'stage1-basic' else cwd / 'stage1-basic'
else:
    # 尝试查找 stage1-basic 目录
    for parent in cwd.parents:
        if parent.name == 'stage1-basic':
            stage1_dir = parent
            break
    if stage1_dir is None:
        # 如果找不到，假设当前目录就是 stage1-basic
        stage1_dir = cwd

# 添加到 sys.path
if stage1_dir and str(stage1_dir) not in sys.path:
    sys.path.insert(0, str(stage1_dir))
    print(f"📁 已添加路径: {stage1_dir}")
else:
    print(f"📁 使用路径: {stage1_dir}")

# 加载环境变量
dotenv.load_dotenv()

# 从 Python 模块导入类（不是从 notebook 文件导入）
from simple_chat import SimpleChatAgent
from memory_chat import MemoryChatAgent, SlidingWindowMemory

print("🔧 环境配置完成")
print(f"Model: {os.getenv('MODEL_NAME', 'qwen-max')}")
print(f"Base URL: {os.getenv('OPENAI_BASE_URL', 'https://api.openai.com/v1')}")
print("✅ 类导入成功：SimpleChatAgent, MemoryChatAgent, SlidingWindowMemory")

📁 已添加路径: d:\BaiduSyncdisk\learning\cursor\myGithub\myAgent\agent-learning\stage1-basic
🔧 环境配置完成
Model: gpt-3.5-turbo
Base URL: https://api.chatanywhere.tech/v1
✅ 类导入成功：SimpleChatAgent, MemoryChatAgent, SlidingWindowMemory


## 🧠 实验1：关键信息提取记忆

滑动窗口策略可能会丢失重要信息。让我们实现一个更智能的记忆策略。

In [2]:
# 定义一个关键信息记忆的类，用于提取和存储用户的关键信息和近期对话
class KeyInfoMemory:
    """
    关键信息提取记忆策略
    保留重要信息，滑动保留最近对话
    """
    
    # 构造函数，初始化窗口大小和必要数据结构
    def __init__(self, window_size=3):
        # 保存滑动窗口大小，类型为int，默认为3
        self.window_size = window_size
        # 初始化关键信息的字典，用于保存用户重要属性
        self.key_info = {}
        # 初始化最近对话的列表，用于保存滑动窗口范围内的对话历史
        self.recent_conversations = []
        # 创建OpenAI客户端，用于与LLM模型进行交互调用
        self.client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),  # 从环境变量获取OpenAI的API密钥
            base_url=os.getenv("OPENAI_BASE_URL") # 从环境变量获取API的基础URL
        )
    
    # 定义从消息内容中提取关键信息的方法
    def extract_key_info(self, message):
        """从消息中提取关键信息"""
        try:
            # 构造prompt，要求大模型仅以JSON格式抽取关键信息，不要其他文字
            # 这是 Python 3.6 及以上支持的 f-string（格式化字符串字面量）语法，
            # 用于在字符串中嵌入变量或表达式。在字符串前加 f，再在字符串内部用 {变量名} 的方式即可将变量值嵌入到字符串里。
            # 支持多行字符串（用三个引号），并能自动格式化变量为字符串。
            #
            # 例如：
            # name = "Alice"
            # age = 18
            # s = f"姓名:{name}, 年龄:{age+1}"
            # print(s)
            # # 输出: 姓名:Alice, 年龄:19
            #
            prompt = f"""
            从以下消息中提取关键信息，只返回JSON格式：
            {{
              "name": "姓名(如果有)",
              "location": "地点(如果有)",
              "job": "职业(如果有)",
              "preference": "偏好/爱好(如果有)"
            }}
            
            消息：{message}
            
            只返回JSON，不要其他文字：
            """
            
            # 通过OpenAI的Chat接口进行请求，设置精度参数（temperature），限制输出最大tokens
            response = self.client.chat.completions.create(
                model=os.getenv("MODEL_NAME", "qwen-max"),  # 指定要调用的模型
                messages=[{"role": "user", "content": prompt}],  # 以对话格式提供prompt
                temperature=0.1,  # 温度低代表答案更稳定
                max_tokens=100    # 限制最多生成的token数量
            )
            
            # 从返回的response结构体里获取模型输出内容，并移除首尾空格
            content = response.choices[0].message.content.strip()
            # 对结果内容做清理，去掉可能的markdown代码块开头和结尾（如```json ... ```）
            content = re.sub(r'```json\n?', '', content).replace('```', '')
            
            # 使用json.loads将字符串解析为Python字典
            info = json.loads(content)
            
            # 遍历抽取出来的信息，如果有值就写入/覆盖当前key_info字典
            for key, value in info.items():
                # 判断value非空且排除"(如果有)"等模板残留
                if value and value.strip() and value != "(如果有)":
                    self.key_info[key] = value.strip()
                    
            # 返回True表示成功
            return True
        except Exception as e:
            # 捕获异常并打印失败原因
            print(f"提取关键信息失败: {e}")
            return False
    
    # 更新记忆的方法，role为"用户"或"助手"，content为消息文本
    def update(self, role, content):
        """更新记忆"""
        # 如果当前消息是用户发的，则尝试提取关键信息
        if role == "user":
            self.extract_key_info(content)
            
        # 将聊天内容以字典形式追加到最近对话列表中
        self.recent_conversations.append({"role": role, "content": content})
        
        # 这里设置最近对话的滑动窗口最大长度为window_size*2，没有加1，
        # 因为每次无论是用户还是助手的消息都会各自append一次，window_size按一轮双人对话计；
        # 如果想一轮包含“用户+助手”消息，可以用*2而不是*2+1。例如window_size=3，则最多保留6条（刚好3轮），无需+1。
        max_recent = self.window_size * 2
        # 如果对话缓存超过窗口长度，则只保留最新N条
        if len(self.recent_conversations) > max_recent:
            # Python的切片[-N:]语法，取最近N条消息
            self.recent_conversations = self.recent_conversations[-max_recent:]
    
    # 获取带有关键信息+近期消息的完整上下文，用作模型输入
    def get_context(self):
        """构建完整的上下文"""
        # 初始化上下文列表
        context = []
        
        # 如果已存有关键信息，则组织系统提示，否则用默认提示
        if self.key_info:
            # 用列表推导和join，将关键信息格式化为"key: value"字符串
            key_info_str = ", ".join([f"{k}: {v}" for k, v in self.key_info.items()])
            # 拼接系统提示，指明用户信息
            system_prompt = f"你是一个有记忆的助手。以下是用户的重要信息：{key_info_str}。记住这些信息并在对话中适当使用。"
        else:
            # 没有关键信息时，用通用友好提示
            system_prompt = "你是一个有记忆的友好助手。"
        
        # 将系统提示作为system角色对话添加到上下文开头
        context.append({"role": "system", "content": system_prompt})
        
        # 用list.extend把最近对话加入上下文（list拼接语法说明）
        context.extend(self.recent_conversations)
        
        # 返回最终上下文
        return context
    
    # 获取当前记忆状态的统计信息
    def get_stats(self):
        """获取统计信息"""
        # 返回一个字典，包括关键信息数、对话数、拷贝的关键信息和窗口大小
        return {
            "key_info_count": len(self.key_info),                    # 关键信息项的数量
            "recent_messages": len(self.recent_conversations),       # 最近窗口的消息数量
            "key_info": self.key_info.copy(),                        # 关键信息的字典（深拷贝）
            "window_size": self.window_size                          # 当前滑动窗口大小
        }

# 定义基于关键信息记忆的聊天助手类
class KeyInfoChatAgent:
    # 构造方法，初始化自己的记忆实例和OpenAI客户端
    def __init__(self, window_size=3):
        # 创建一个KeyInfoMemory实例，用于内部记忆管理
        self.memory = KeyInfoMemory(window_size)
        # 创建OpenAI客户端，用于后续与大模型接口交互
        self.client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),  # 获取API_KEY
            base_url=os.getenv("OPENAI_BASE_URL") # 获取基础URL
        )
    
    # 聊天方法，输入用户消息，返回回复
    def chat(self, message):
        try:
            # 更新记忆：把本轮用户消息写入内部记忆
            self.memory.update("user", message)
            
            # 获取用于上下文的最近聊天历史+关键信息
            context = self.memory.get_context()
            
            # 向OpenAI接口发起请求，发送完整上下文
            response = self.client.chat.completions.create(
                model=os.getenv("MODEL_NAME", "qwen-max"),  # 指定聊天模型名称
                messages=context,                           # 上下文消息历史
                temperature=0.3,                            # 生成回答的多样性
                max_tokens=300                              # 限制最大生成token数
            )
            
            # 获取大模型返回的助手消息文本
            assistant_msg = response.choices[0].message.content
            # 把助手的回复写入记忆（角色为assistant）
            self.memory.update("assistant", assistant_msg)
            
            # 返回一个结构体，包含回复内容、消耗token数和当前记忆统计信息
            return {
                "response": assistant_msg,
                "tokens_used": response.usage.total_tokens,
                "memory_stats": self.memory.get_stats()
            }
        except Exception as e:
            # 捕获错误，返回异常说明
            return {"error": str(e)}

# 打印关键信息记忆系统已创建，用于调试与提示
print("✅ 关键信息记忆系统已创建")

✅ 关键信息记忆系统已创建


### 🧪 对比三种记忆策略

In [ ]:
# 测试对话序列
test_conversation = [
    "你好，我叫李华，是一名软件工程师。",
    "我住在深圳南山，喜欢打篮球和看书。",
    "我正在学习AI技术，对Agent开发很感兴趣。",
    "我的目标是成为一名AI专家。",
    "你觉得我应该从哪里开始学起？",
    "对了，我刚才说我叫什么名字？",
    "我住在哪个城市？",
    "我的职业是什么？",
    "我有什么爱好？",
    "请给我一些学习建议。"
]

print("🔍 对比三种记忆策略")
print("=" * 60)

# 1. 简单助手（无记忆）
print("\n🤖 1. 简单助手（无记忆）")
print("-" * 40)
simple_agent = SimpleChatAgent()
simple_results = []
for i, msg in enumerate(test_conversation[-5:], 6):  # 只测试后5个问题
    result = simple_agent.chat(msg)
    simple_results.append(result)
    print()

# 2. 滑动窗口记忆
print("\n🧠 2. 滑动窗口记忆")
print("-" * 40)
window_agent = MemoryChatAgent(window_size=3)
window_results = []
for msg in test_conversation:
    result = window_agent.chat(msg)
    window_results.append(result)
    if "名字" in msg or "城市" in msg or "职业" in msg or "爱好" in msg:
        # 修复：先判断memory_stats的类型（避免TypeError）
        stats = result.get('memory_stats')
        if isinstance(stats, dict) and 'total_messages' in stats:
            print(f"📊 记忆状态: {stats['total_messages']}条消息")
        else:
            print("📊 记忆状态: （无统计数据）")
    print()

# 3. 关键信息记忆
print("\n🎯 3. 关键信息记忆")
print("-" * 40)
keyinfo_agent = KeyInfoChatAgent(window_size=3)
keyinfo_results = []
for msg in test_conversation:
    result = keyinfo_agent.chat(msg)
    keyinfo_results.append(result)
    if "名字" in msg or "城市" in msg or "职业" in msg or "爱好" in msg:
        stats = result.get('memory_stats')
        if isinstance(stats, dict) and 'key_info' in stats and 'recent_messages' in stats:
            print(f"📊 关键信息: {stats['key_info']}, 记忆消息: {stats['recent_messages']}")
        else:
            print("📊 关键信息: （无统计数据）")
    print()

🔍 对比三种记忆策略

🤖 1. 简单助手（无记忆）
----------------------------------------






🧠 2. 滑动窗口记忆
----------------------------------------






## 📊 性能对比分析

In [ ]:
# 计算性能指标
def calculate_metrics(results):
    tokens = [r.get('tokens_used', 0) for r in results if 'tokens_used' in r]
    return {
        'total_tokens': sum(tokens),
        'avg_tokens': sum(tokens) / len(tokens) if tokens else 0,
        'conversation_count': len(results)
    }

simple_metrics = calculate_metrics(simple_results)
window_metrics = calculate_metrics(window_results)
keyinfo_metrics = calculate_metrics(keyinfo_results)

# 创建对比表格
comparison_data = {
    "策略": ["无记忆", "滑动窗口", "关键信息"],
    "总Token": [simple_metrics['total_tokens'], window_metrics['total_tokens'], keyinfo_metrics['total_tokens']],
    "平均Token/次": [simple_metrics['avg_tokens'], window_metrics['avg_tokens'], keyinfo_metrics['avg_tokens']],
    "记忆持久性": ["无", "短期", "长期"],
    "信息保留": ["低", "中等", "高"]
}

df = pd.DataFrame(comparison_data)
print("📈 性能对比分析:")
print(df.to_string(index=False))

# 可视化对比
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# 1. 总Token使用对比
ax1.bar(comparison_data["策略"], comparison_data["总Token"], color=['lightgray', 'skyblue', 'lightgreen'])
ax1.set_title("总Token使用对比")
ax1.set_ylabel("Token数量")

# 2. 平均Token使用对比
ax2.bar(comparison_data["策略"], comparison_data["平均Token/次"], color=['lightgray', 'skyblue', 'lightgreen'])
ax2.set_title("平均Token使用/次对比")
ax2.set_ylabel("Token数量")

# 3. 记忆持久性评分
memory_scores = [1, 3, 5]  # 无记忆=1, 短期=3, 长期=5
ax3.bar(comparison_data["策略"], memory_scores, color=['lightgray', 'skyblue', 'lightgreen'])
ax3.set_title("记忆持久性评分")
ax3.set_ylabel("评分 (1-5)")
ax3.set_ylim(0, 6)

# 4. 信息保留评分
info_scores = [1, 3, 5]  # 低=1, 中等=3, 高=5
ax4.bar(comparison_data["策略"], info_scores, color=['lightgray', 'skyblue', 'lightgreen'])
ax4.set_title("信息保留评分")
ax4.set_ylabel("评分 (1-5)")
ax4.set_ylim(0, 6)

plt.tight_layout()
plt.show()

## 🎮 互动实验区

In [ ]:
# 在这里进行你自己的实验！
# 尝试不同的对话场景，测试记忆策略的效果

# 示例：测试长对话中的信息保持
long_dialogue = [
    "我是王教授，在清华大学教计算机科学。",
    "我研究人工智能已经有15年了。",
    "我主要关注深度学习和自然语言处理。",
    "我写过3本书，发表了50多篇论文。",
    "我指导了20多个研究生。",
    "我的爱好是古典音乐和围棋。",
    "我会弹钢琴，也喜欢爬山。",
    "我有一个女儿正在上高中。",
    "我太太是一名医生。",
    "我住在清华大学的教师公寓。"
]

test_questions = [
    "我叫什么名字？",
    "我在哪里工作？",
    "我的研究领域是什么？",
    "我有什么爱好？",
    "我写了多少本书？"
]

print("🧪 长对话信息保持测试")
print("=" * 40)

# 选择你想要测试的策略
print("选择测试策略 (1-3):")
print("1. 关键信息记忆")
print("2. 滑动窗口记忆")
print("3. 简单助手")

# 取消注释以下代码进行互动测试
choice = input("请输入选择: ")

if choice == "1":
    agent = KeyInfoChatAgent(window_size=3)
    strategy = "关键信息记忆"
elif choice == "2":
    agent = MemoryChatAgent(window_size=5)
    strategy = "滑动窗口记忆"
else:
    agent = SimpleChatAgent()
    strategy = "简单助手"

print(f"\n🎯 使用策略: {strategy}")
print("-" * 30)

# 先进行长对话
for msg in long_dialogue:
    agent.chat(msg)

# 再测试记忆保持
for question in test_questions:
    print(f"\n问题: {question}")
    result = agent.chat(question)
    print(f"回答: {result.get('response', result.get('error', 'Unknown error'))}")

print("\n💡 提示：取消注释上面的代码进行互动测试！")

## 📝 实验总结与思考

### 🔍 实验发现

通过上面的对比实验，你可能发现：

1. **无记忆助手**：每次对话都是独立的，无法保持上下文
2. **滑动窗口记忆**：能保持最近对话，但可能丢失早期重要信息
3. **关键信息记忆**：能长期保存重要信息，但需要额外的Token开销

### ⚖️ 权衡考虑

| 策略 | 优点 | 缺点 | 适用场景 |
|------|------|------|----------|
| 无记忆 | Token少、响应快 | 无记忆能力 | 简单问答 |
| 滑动窗口 | 平衡性好 | 可能丢失信息 | 短期对话 |
| 关键信息 | 信息保持好 | Token开销大 | 长期关系 |

### 🚀 优化方向

1. **混合策略**：结合滑动窗口和关键信息提取
2. **动态窗口**：根据对话复杂度调整窗口大小
3. **智能压缩**：用更少Token表达相同信息
4. **分层记忆**：区分短期记忆和长期记忆

### 🎯 实践建议

- 短对话：使用滑动窗口（window_size=3-5）
- 长期关系：使用关键信息记忆
- 成本敏感：减小窗口大小或使用混合策略
- 信息密集：增加窗口大小或使用关键信息提取

### 🤔 思考题

1. 如何评估信息的重要性？
2. 怎样在保持记忆和控制成本之间找到最佳平衡？
3. 还有哪些创新的记忆管理策略？

## 🎉 实验完成！

恭喜你完成了Agent记忆优化的深入实验！现在你对Agent的记忆管理有了更深入的理解。下一阶段将学习如何让Agent调用外部工具，敬请期待！